In [1]:
import pandas as pd
import numpy as np

from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
# 정답과 예측을 넣으면, 정확도를 산출해서 보여준다.
from sklearn.metrics import accuracy_score
# 평가지표를 한눈에 보게해주는 라이브러리
from sklearn.metrics import classification_report

In [2]:
baseData = pd.read_csv("../../dataset/keggledata.csv")

In [3]:
minSiu = min(baseData.salary_in_usd)
maxSiu = max(baseData.salary_in_usd)
print(minSiu ,"~", maxSiu)

5132 ~ 450000


In [4]:
labelEn = LabelEncoder()

encodeData = baseData.copy()

encodeData["work_year2"] = labelEn.fit_transform(encodeData.work_year)
encodeData = encodeData.drop(columns=["work_year"])

encodeData["experience_level2"] = labelEn.fit_transform(encodeData.experience_level)
encodeData = encodeData.drop(columns=["experience_level"])

encodeData["employment_type2"] = labelEn.fit_transform(encodeData.employment_type)
encodeData = encodeData.drop(columns=["employment_type"])

encodeData["job_title2"] = labelEn.fit_transform(encodeData.job_title)
encodeData = encodeData.drop(columns=["job_title"])

encodeData["salary_currency2"] = labelEn.fit_transform(encodeData.salary_currency)
encodeData = encodeData.drop(columns=["salary_currency"])

encodeData["employee_residence2"] = labelEn.fit_transform(encodeData.employee_residence)
encodeData = encodeData.drop(columns=["employee_residence"])

encodeData["company_location2"] = labelEn.fit_transform(encodeData.company_location)
encodeData = encodeData.drop(columns=["company_location"])

encodeData["company_size2"] = labelEn.fit_transform(encodeData.company_size)
encodeData = encodeData.drop(columns=["company_size"])
encodeData

,salary,salary_in_usd,remote_ratio,work_year2,experience_level2,employment_type2,job_title2,salary_currency2,employee_residence2,company_location2,company_size2
0,80000,85847,100,3,3,2,84,7,26,25,0
1,30000,30000,100,3,2,0,66,19,75,70,2
2,25500,25500,100,3,2,0,66,19,75,70,2
3,175000,175000,100,3,3,2,47,19,11,12,1
4,120000,120000,100,3,3,2,47,19,11,12,1
...,...,...,...,...,...,...,...,...,...,...,...
3750,412000,412000,100,0,3,2,47,19,75,70,0
3751,151000,151000,100,1,2,2,84,19,75,70,0
3752,105000,105000,100,0,0,2,47,19,75,70,2
3753,100000,100000,100,0,0,0,17,19,75,70,0


In [5]:
encodeData["salary_in_usd"] = np.where(encodeData.salary_in_usd<100000,1,
                                 np.where(encodeData.salary_in_usd<200000,2,
                                         np.where(encodeData.salary_in_usd<300000,3,
                                                 np.where(encodeData.salary_in_usd<400000,4,5))))

encodeData.salary_in_usd.drop_duplicates()
encodeData

,salary,salary_in_usd,remote_ratio,work_year2,experience_level2,employment_type2,job_title2,salary_currency2,employee_residence2,company_location2,company_size2
0,80000,1,100,3,3,2,84,7,26,25,0
1,30000,1,100,3,2,0,66,19,75,70,2
2,25500,1,100,3,2,0,66,19,75,70,2
3,175000,2,100,3,3,2,47,19,11,12,1
4,120000,2,100,3,3,2,47,19,11,12,1
...,...,...,...,...,...,...,...,...,...,...,...
3750,412000,5,100,0,3,2,47,19,75,70,0
3751,151000,2,100,1,2,2,84,19,75,70,0
3752,105000,2,100,0,0,2,47,19,75,70,2
3753,100000,2,100,0,0,0,17,19,75,70,0


In [6]:
corrDf = encodeData.corr()
stdRatio = 0.2
corrDf

,salary,salary_in_usd,remote_ratio,work_year2,experience_level2,employment_type2,job_title2,salary_currency2,employee_residence2,company_location2,company_size2
salary,1.000000,-0.018264,0.028731,-0.094724,-0.024884,-0.003908,0.039065,-0.091121,-0.087176,-0.072774,-0.100352
salary_in_usd,-0.018264,1.000000,-0.073896,0.208519,0.282274,-0.009241,0.104721,0.424435,0.402268,0.395309,-0.007345
remote_ratio,0.028731,-0.073896,1.000000,-0.236430,-0.054025,-0.028673,-0.042451,-0.046644,-0.083142,-0.071574,-0.036928
work_year2,-0.094724,0.208519,-0.236430,1.000000,0.187748,0.011127,-0.026862,0.254548,0.243475,0.234661,0.138875
experience_level2,-0.024884,0.282274,-0.054025,0.187748,1.000000,-0.032794,0.020026,0.225464,0.251325,0.250591,0.066414
employment_type2,-0.003908,-0.009241,-0.028673,0.011127,-0.032794,1.000000,-0.052231,-0.048446,-0.024582,-0.013963,-0.041001
job_title2,0.039065,0.104721,-0.042451,-0.026862,0.020026,-0.052231,1.000000,-0.057072,-0.063911,-0.073549,0.002548
salary_currency2,-0.091121,0.424435,-0.046644,0.254548,0.225464,-0.048446,-0.057072,1.000000,0.755836,0.764744,0.102738
employee_residence2,-0.087176,0.402268,-0.083142,0.243475,0.251325,-0.024582,-0.063911,0.755836,1.000000,0.945676,0.032168
company_location2,-0.072774,0.395309,-0.071574,0.234661,0.250591,-0.013963,-0.073549,0.764744,0.945676,1.000000,0.041703


In [7]:
feature = list(corrDf.loc[(abs(corrDf.salary_in_usd)>stdRatio)&(corrDf.salary_in_usd!=1)].index)
feature

['work_year2',
 'experience_level2',
 'salary_currency2',
 'employee_residence2',
 'company_location2']

In [8]:
label = ["salary_in_usd"]

In [9]:
featuredData = encodeData.loc[:,feature]
featuredData

,work_year2,experience_level2,salary_currency2,employee_residence2,company_location2
0,3,3,7,26,25
1,3,2,19,75,70
2,3,2,19,75,70
3,3,3,19,11,12
4,3,3,19,11,12
...,...,...,...,...,...
3750,0,3,19,75,70
3751,1,2,19,75,70
3752,0,0,19,75,70
3753,0,0,19,75,70


In [10]:
minMaxData = featuredData.copy()

In [11]:
minMaxData.work_year2 = minmax_scale(minMaxData.work_year2)
minMaxData.experience_level2 = minmax_scale(minMaxData.experience_level2)
minMaxData.salary_currency2 = minmax_scale(minMaxData.salary_currency2)
minMaxData.employee_residence2 = minmax_scale(minMaxData.employee_residence2)
minMaxData.company_location2 = minmax_scale(minMaxData.company_location2)
minMaxData

,work_year2,experience_level2,salary_currency2,employee_residence2,company_location2
0,1.000000,1.000000,0.368421,0.337662,0.352113
1,1.000000,0.666667,1.000000,0.974026,0.985915
2,1.000000,0.666667,1.000000,0.974026,0.985915
3,1.000000,1.000000,1.000000,0.142857,0.169014
4,1.000000,1.000000,1.000000,0.142857,0.169014
...,...,...,...,...,...
3750,0.000000,1.000000,1.000000,0.974026,0.985915
3751,0.333333,0.666667,1.000000,0.974026,0.985915
3752,0.000000,0.000000,1.000000,0.974026,0.985915
3753,0.000000,0.000000,1.000000,0.974026,0.985915


In [12]:
labelData = encodeData.loc[:,label]
labelData

,salary_in_usd
0,1
1,1
2,1
3,2
4,2
...,...
3750,5
3751,2
3752,2
3753,2


In [13]:
print(featuredData.shape)
print(labelData.shape)

(3755, 5)
(3755, 1)


In [14]:
trainDataFeat,\
testDataFeat,\
trainDataLabel,\
testDataLabel = train_test_split(minMaxData, labelData, test_size=0.3, random_state=10)

In [15]:
testIndex = testDataLabel.index

In [16]:
print(trainDataFeat.shape)
print(trainDataLabel.shape)
print(testDataFeat.shape)
print(testDataLabel.shape)

(2628, 5)
(2628, 1)
(1127, 5)
(1127, 1)


In [17]:
model_mrthod = svm.SVC(random_state=10)

In [18]:
model = model_mrthod.fit(X = trainDataFeat,
                         y = trainDataLabel)

C:\Users\WIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
predict = pd.DataFrame(model.predict(X = testDataFeat), columns=["predict"])

In [20]:
testDataLabel.reset_index(drop=True, inplace=True)

In [21]:
resultData = pd.concat([testDataLabel,predict],axis=1)

In [22]:
testDataFeat.reset_index(drop=True, inplace=True)

In [23]:
resultData2 = pd.concat([testDataFeat,resultData],axis=1)
resultData2

,work_year2,experience_level2,salary_currency2,employee_residence2,company_location2,salary_in_usd,predict
0,0.666667,1.000000,1.0,0.974026,0.985915,2,2
1,1.000000,1.000000,1.0,0.974026,0.985915,3,2
2,1.000000,0.666667,1.0,0.974026,0.985915,3,2
3,1.000000,1.000000,1.0,0.974026,0.985915,2,2
4,0.666667,0.666667,1.0,0.974026,0.985915,2,2
...,...,...,...,...,...,...,...
1122,0.000000,0.666667,1.0,0.766234,0.985915,1,2
1123,0.666667,1.000000,1.0,0.974026,0.985915,1,2
1124,1.000000,1.000000,1.0,0.974026,0.985915,2,2
1125,1.000000,1.000000,1.0,0.974026,0.985915,2,2


In [24]:
feature

['work_year2',
 'experience_level2',
 'salary_currency2',
 'employee_residence2',
 'company_location2']

In [25]:
feature2 = []
for i in range(0,len(feature)):
    featCol = feature[i].split("2")
    feature2.append(featCol[0])
feature2

['work_year',
 'experience_level',
 'salary_currency',
 'employee_residence',
 'company_location']

In [26]:
baseData

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [27]:
concatData = baseData.loc[testIndex,feature2]
concatData.reset_index(drop = True, inplace = True)
concatData

,work_year,experience_level,salary_currency,employee_residence,company_location
0,2022,SE,USD,US,US
1,2023,SE,USD,US,US
2,2023,MI,USD,US,US
3,2023,SE,USD,US,US
4,2022,MI,USD,US,US
...,...,...,...,...,...
1122,2020,MI,USD,PH,US
1123,2022,SE,USD,US,US
1124,2023,SE,USD,US,US
1125,2023,SE,USD,US,US


In [28]:
finalData = pd.concat([concatData,resultData], axis = 1)
finalData
# salary_in_usd가 숫자가 기니까 너무 값이 많이 나오는 것 같다.
# 돈값의 범위를 지정해서 학습시키는게 어떨까?

,work_year,experience_level,salary_currency,employee_residence,company_location,salary_in_usd,predict
0,2022,SE,USD,US,US,2,2
1,2023,SE,USD,US,US,3,2
2,2023,MI,USD,US,US,3,2
3,2023,SE,USD,US,US,2,2
4,2022,MI,USD,US,US,2,2
...,...,...,...,...,...,...,...
1122,2020,MI,USD,PH,US,1,2
1123,2022,SE,USD,US,US,1,2
1124,2023,SE,USD,US,US,2,2
1125,2023,SE,USD,US,US,2,2


In [29]:
accuracy_score(finalData.salary_in_usd, finalData.predict)

0.6974267968056788

In [30]:
print(classification_report(finalData.salary_in_usd, finalData.predict))
print(accuracy_score(finalData.salary_in_usd, finalData.predict))

              precision    recall  f1-score   support

           1       0.75      0.61      0.68       298
           2       0.68      0.92      0.78       657
           3       0.00      0.00      0.00       150
           4       0.00      0.00      0.00        20
           5       0.00      0.00      0.00         2

    accuracy                           0.70      1127
   macro avg       0.29      0.31      0.29      1127
weighted avg       0.60      0.70      0.64      1127

0.6974267968056788


C:\Users\WIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\WIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\WIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
feature2

['work_year',
 'experience_level',
 'salary_currency',
 'employee_residence',
 'company_location']

# 예측하고 싶은 값을 구하기 위한 튜닝

In [32]:
feature

['work_year2',
 'experience_level2',
 'salary_currency2',
 'employee_residence2',
 'company_location2']

In [33]:
minMaxData.work_year2.drop_duplicates()

0      1.000000
41     0.666667
183    0.000000
776    0.333333
Name: work_year2, dtype: float64

In [34]:
minMaxData.experience_level2.drop_duplicates()

0     1.000000
1     0.666667
13    0.000000
58    0.333333
Name: experience_level2, dtype: float64

In [35]:
minMaxData.salary_currency2.drop_duplicates()

0       0.368421
1       1.000000
41      0.631579
80      0.473684
114     0.157895
157     0.421053
218     0.000000
219     0.842105
322     0.105263
528     0.578947
972     0.052632
998     0.894737
1165    0.789474
2358    0.526316
2684    0.263158
2923    0.315789
3475    0.684211
3594    0.736842
3597    0.947368
3669    0.210526
Name: salary_currency2, dtype: float64

In [36]:
minMaxData.employee_residence2.drop_duplicates()

0       0.337662
1       0.974026
3       0.142857
21      0.259740
23      0.376623
          ...   
3638    0.844156
3657    0.753247
3720    0.662338
3724    0.623377
3748    0.688312
Name: employee_residence2, Length: 78, dtype: float64

In [37]:
minMaxData.company_location2.drop_duplicates()

0       0.352113
1       0.985915
3       0.169014
21      0.281690
23      0.394366
          ...   
3621    0.225352
3657    0.774648
3669    0.211268
3720    0.676056
3748    0.704225
Name: company_location2, Length: 72, dtype: float64

In [38]:
inputWork = 0.333333
inputExlevel = 0.842105
inputSalaryCu = 0.842105
inputEmployResi = 0.844156
inputCompany = 0.985915

In [39]:
testData = pd.DataFrame([[inputWork, inputExlevel, inputSalaryCu, inputEmployResi, inputCompany]])
predictResult = model.predict(testData)
finalPredict = predictResult[0]
print(finalPredict)

1


C:\Users\WIN\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [40]:
testDataLabel = testDataLabel.rename(columns={"salary_in_usd":"salary_in_usd_labeling"})

In [41]:
check = baseData.salary_in_usd[testIndex]
check.reset_index(drop=True, inplace=True)
checkData = pd.concat([check,testDataLabel], axis = 1)

In [42]:
scData = baseData.salary_currency[testIndex]

In [43]:
scData.reset_index(drop=True, inplace=True)

In [44]:
finalData = pd.concat([checkData,scData], axis=1)

In [45]:
result = finalData.loc[finalData.salary_in_usd_labeling==finalPredict]

# 최종 급여의 범위

In [46]:
minRe = min(result.salary_in_usd)
maxRe = max(result.salary_in_usd)
print(minRe,"~",maxRe)

5132 ~ 99703
